In [ ]:
# This notebook is derived from Qiskit and includes modifications by qBraid.
#
# (C) Copyright IBM 2020.
# (C) Copyright qBraid 2023.
#
# This code is licensed under the Apache License, Version 2.0. You may
# obtain a copy of this license in the LICENSE.txt file in the root directory
# of this source tree or at http://www.apache.org/licenses/LICENSE-2.0.
#
# Any modifications or derivative works of this code must retain this
# copyright notice, and modified files need to carry a notice indicating
# that they have been altered from the originals.

# qBraid-SDK Qiskit on AWS Device Demo: Bernstein-Vazirani Algorithm

In [ ]:
import qbraid

qbraid.__version__

## Creating the Circuit

In [ ]:
from qiskit import QuantumCircuit

The code for this circuit was taken from IBMs Bernstein-Vazirani algorithm tutorial. Check out their [tutorial](https://www.youtube.com/watch?v=sqJIpHYl7oo) for a more in depth explanation. Classically, it takes $n$ queries to decipher a secret string of length $n$ – the Bernstein-Vazirani algorithm allows us to develop an oracle in a quantum circuit that is able to guess the string with just one query!

In [ ]:
s = "110101"  # the number we hope to guess
n = len(s)

qiskit_circuit = QuantumCircuit(n + 1, n)

qiskit_circuit.x(n)
qiskit_circuit.barrier()
qiskit_circuit.h(range(n + 1))
qiskit_circuit.barrier()

for ii, yesno in enumerate(reversed(s)):
    if yesno == "1":
        qiskit_circuit.cx(ii, n)

qiskit_circuit.barrier()
qiskit_circuit.h(range(n + 1))
qiskit_circuit.barrier()
qiskit_circuit.measure(range(n), range(n))

In [ ]:
from qbraid.visualization import circuit_drawer

circuit_drawer(qiskit_circuit, "mpl")  # Visualizing the circuit

## Running on an AWS Device via qBraid

Let's check which devices (specifically AWS ones) are online, and also find their device IDs:

In [ ]:
from qbraid.runtime.braket import BraketProvider

In [ ]:
provider = BraketProvider()
provider.get_devices()

In this tutorial we'll use Amazon's SV1 computer, since we see that it's online. Now we can use the BraketProvider run a job on an Amazon device. The device wrapper adds a layer of abstraction, allowing us to run a qiskit circuit on an AWS quantum computer. Note that there's no need for any sort of circuit wrapper here – we can plug the qiskit circuit directly into the wrapped device!

In [ ]:
aws_device_id = "arn:aws:braket:::device/quantum-simulator/amazon/sv1" 
device = provider.get_device(aws_device_id)

In [ ]:
braket_job = device.run(qiskit_circuit, shots=100)
braket_job.status()

In [ ]:
result = braket_job.result()
counts = result.measurement_counts()
print(counts)

In [ ]:
from qbraid.visualization import plot_histogram

In [ ]:
plot_histogram(counts)

We have guessed the number correctly! Note the bar graph where only one number was guessed in all 100 shots